<a href="https://colab.research.google.com/github/Yewon-dev/boostcamp-AI-Tech/blob/main/TIL/%5B%EC%8B%AC%ED%99%94%EB%AC%B8%EC%A0%9C%5DTransfer_Learning_%26_Hyper_Parameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning and Hyperparameter Tuning

🚀 목표 : CNN에서 **Transfer Learning** (Fine-Tune, torchvision)과 **Hyperparameter Tuning**을 위한 PyTorch 및 추가 라이브러리(*Ray Tune*) 사용법 익히기



### **실습 내용**

Fashion-Mnist라는 10개의 의류 종류를 포함한 데이터셋을 분류하는 딥러닝 방법론 중 하나인 **Convolution Neural Networks** 모델을 **Transfer Learning**으로 학습하기


----

## Transfer Learning

Transfer Learning은 "Source Tasks"에서 학습된 지식을 "Target Task"로 전이하는 절차 및 방법론이다.

### 1. Source Task 모델 만들기

### 1.1 ImangeNet Pretrained Model을 torchvidion에서 불러오기

- ImageNet에서 학습된 **ResNet 18** 딥러닝 모델을 불러옴


In [ ]:
imagenet_resnet18 = torchvision.models.resnet18(pretrained=True)

### 1.2 Mnist Pretrained Model 만들기

#### 1.2.1 Mnist Dataset 불러오기

- Mnist Dataset은 0부터 9까지 손으로 쓰인 10가지의 클래스가 있는 데이터셋

In [ ]:
mnist_train = torchvision.datasets.MNIST(root='./mnist', train=True, download=True)
mnist_test = torchvision.datasets.MNIST(root='./mnist', train=False, download=True)

#### 1.2.2 Mnist를 학습할 CNN 모델 생성 (ResNet18)

In [ ]:
mnist_resnet18 = torchvision.models.resnet18(pretrained=False)

#### 1.2.3 Mnist 데이터 분류 ResNet18 모델에 학습하기

*   torchvision.datasets.minist의 데이터 타입은 PIL Image -> 학습 때는 Tensor type으로 변환해야 한다
*   원본 영상은 gray scale이지만 모델은 3채널이므로 gray scale을 RGB로 변환해야 한다
  - Q. 모델 입력을 channel 1개만 받도록 변경한다면?
  - `mnist_resnet18.conv1 = nn.Conv2d(in_channels = 1, ...)`




In [ ]:
common_transform = torchvision.transforms.Compose([
    # grayscale의 1채널 영상을 3채널로 동일한 값으로 확장함
    torchvision.transforms.Grayscale(num_output_channels=3), 
    # PIL Image를 Tensor type로 변경함
    torchvision.transforms.ToTensor() 
  ])

(결과)
```
원본 <class 'PIL.Image.Image'> (28, 28)
변경됨 <class 'torch.Tensor'> (3, 28, 28)
```





---



**Xavier Initialization**

Xavier Initialization 혹은 Glorot Initialization라고도 불리는 초기화 방법은 **이전 노드와 다음 노드의 개수에 의존**하는 방법이다. Uniform 분포를 따르는 방법과 Normal분포를 따르는 두가지 방법이 사용된다.(Glorot & Bengio, AISTATS 2010)

- Xavier Normal Initialization
$$ W∼N(0,Var(W)) $$
$$ Var(W)= \sqrt{\frac{2}{n_{in}+n_{out}}} $$
($n_{in}$ : 이전 layer(input)의 노드 수, $n_{out}$ : 다음 layer의 노드 수)

- Xavier Uniform Initialization
$$ W∼U(-\sqrt{\frac{6}{n_{in}+n_{out}}},+\sqrt{\frac{6}{n_{in}+n_{out}}}) $$
($n_{in}$ : 이전 layer(input)의 노드 수, $n_{out}$ : 다음 layer의 노드 수)

Xaiver함수는 비선형함수(ex. sigmoid, tanh)에서 효과적인 결과를 보여준다.

(참고) https://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf

---



**tqdm**

`from tqdm.notebook import tqdm`

- tqdm이라는 "반복문"의 현재 진행 상태를 progress-bar로 보여주는 라이브러리

(참고) https://github.com/tqdm/tqdm




---



**.eval()**


`model.eval()`
- `nn.Module`에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
- evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off시킴



---



`for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):`
- progress bar 나타내기 위함
- Q. with 과 pbar을 사용하용 epoch, running loss 등 나타내기



----

### 1.3 Target Task 모델 학습하기



#### 1.3.1 Fashion-Mnist Dataset 불러오기

- ref - https://github.com/zalandoresearch/fashion-mnist

In [ ]:
fashion_train = torchvision.datasets.FashionMNIST(root='./fashion', train=True, download=True)
fashion_test = torchvision.datasets.FashionMNIST(root='./fashion', train=False, download=True)

#### 1.3.2 Fashion-Mnist를 학습할 Source Task 모델 가져오기
- 앞서 (1.2.2)에서 imagenet으로 사전 학습된 ResNet18을 만듦

In [ ]:
target_model = imagenet_resnet18
FASHION_INPUT_NUM = 1
FASHION_CLASS_NUM = 10

## imagenet_resnet18 은 필요 입력 채널 개수가 3, 예측하는 클래스 종류는 1000가지
## Fashion Mnist 데이터는 필요 입력 채널 개수가 1, 레이블은 10가지 타입
## 따라서 Fine-Tuning 전에 모델 구조 변경
target_model.conv1 = torch.nn.Conv2d(FASHION_INPUT_NUM, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
target_model.fc = torch.nn.Linear(in_features=512, out_features=FASHION_CLASS_NUM, bias=True)


#### 1.3.3 Fashion-Mnist 데이터 분류 학습

In [ ]:
# 앞서 선언한 데이터셋에 transform 인자를 넘겨주자
fashion_train_transformed = torchvision.datasets.FashionMNIST(root='./fashion', train=True, download=True, transform=common_transform)
fashion_test_transformed = torchvision.datasets.FashionMNIST(root='./fashion', train=False, download=True, transform=common_transform)

# Mnist Dataset을 DataLoader에 붙이기
BATCH_SIZE = 64
fashion_train_dataloader = torch.utils.data.DataLoader(fashion_train_transformed, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
fashion_test_dataloader = torch.utils.data.DataLoader(fashion_test_transformed, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

- mnist train 데이터셋을 ResNet18 모델에 학습

In [ ]:
LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 5 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross entropy loss를 objective function으로 사용 - https://en.wikipedia.org/wiki/Cross_entropy
optimizer = torch.optim.Adam(target_model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함


In [ ]:
[학습코드]



---



## Hyper Parameter Tuning
- Ray : Distributed application을 만들기 위한 프레임워크
- Tune : Ray 안에 있는 라이브러리.

(참고) https://docs.ray.io/en/master/tune/index.html

### 1. Ray Tune 사용하기

- Ray 프레임워크 설치
```python
!pip uninstall -y -q pyarrow
!pip install -q -U ray[tune]
!pip install -q ray[debug]
```